In [17]:
import shutil
import sys
import os
from pathlib import Path
import json
import re

In [18]:
root_path = Path('/home/jianghui/dataset/Hierachal-MovieNet/')
img_root_path = Path(root_path, 'all/img/')
shot_root_path = Path(root_path, 'all/shot/')
scene_root_path = Path(root_path, 'all/scene/')
subtitle_root_path = Path(root_path, 'all/subtitle/')
synopsis_root_path = Path(root_path, 'all/synopsis/')
cast_root_path = Path(root_path, 'all/cast/')
subtitle_img_root_path = Path(root_path, 'all/subtitle_img/')
names = locals()

In [19]:
dest_path = Path('/home/jianghui/dataset/VideoReorder-MovieNet/')
dest_train_path = Path(dest_path, 'train/')


In [20]:
tt_list = []
with open(Path(root_path, 'movie_id_list'), 'r') as f_tt_list:
    for tt_line in f_tt_list:
        tt_list.append(tt_line.replace('\n', ''))

In [21]:
def open_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    return names[f'{types}_json']

def save_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)

In [31]:
DEBUG = False

img_group_idx = 0
total_num = 0

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')
    shot_json = open_json('shot', tt_id)

    img_num = 0
    img_list = []

    # gain img select in shot
    img_select_in_shot = []
    for shot_ele in shot_json:
        if shot_ele['subtitle_id'] == []:
            subtitle_num_in_shot = 0
        else:
            subtitle_num_in_shot = shot_ele['subtitle_id'][1] - shot_ele['subtitle_id'][0] + 1
            if subtitle_num_in_shot > 3: subtitle_num_in_shot = 3

        img_select_in_shot.append(subtitle_num_in_shot)

    img_list = []
    img_list_ele = []
    shot_num = len(shot_json)

    i = 0
    while i < shot_num - 2:

        if img_select_in_shot[i] == 0: 
            i += 1
            continue
        img_num = 0
        
        j = i + 2
        tmp_rate = sum(img_select_in_shot[i:j+1]) / 9.0
        if tmp_rate < 0.67:
            i += 1
            continue

        j += 1
        last_j = 0
        while j < shot_num:

            tmp_rate = sum(img_select_in_shot[i:j+1]) / (3 * (j+1-i))
            if tmp_rate < 0.67:
                last_j = j - 1
                break
            
            if j+1-i > 7:
                last_j = j - 1
                break

            j += 1
        if last_j == 0: last_j = shot_num - 1

        img_list_ele = [i, last_j]
        img_list.append(img_list_ele)
        
        i = j

    if DEBUG:
        print("img_list is \n",img_list)

    total_num += len(img_list)
    print(f'{tt_id} has group {len(img_list)} and total_num is {total_num}')

    for idx, img_list_ele in enumerate(img_list):
        file_name = img_group_idx
        img_group_idx += 1
        file_path = Path(dest_train_path, str(file_name))
        Path.mkdir(file_path, exist_ok=True)

        img_num = (img_list_ele[1] + 1 - img_list_ele[0]) * 3

        # generate img
        for i, shot_id in enumerate(range(img_list_ele[0], img_list_ele[1] + 1)):
            
            # generate img old name and new name
            for j in range(3):
                old_img_name = f'{shot_id}_{j}.jpg'
                old_img_path = Path(img_root_path, tt_id, old_img_name)
                new_img_name = f'{file_name}_{i*3 + j}.jpg'
                new_img_path = Path(file_path, new_img_name)
                shutil.copyfile(old_img_path, new_img_path)

        old_subtitle_file_path = Path(subtitle_root_path, tt_id + ".json")
        with open(old_subtitle_file_path, 'r') as f_old_subtitle_file:
            old_subtitle_json = json.load(f_old_subtitle_file)

        # generate and save subtitle.json
        new_subtitle_json = [[] for i in range(img_num)]
        for shot_id_idx, shot_id in enumerate(range(img_list_ele[0], img_list_ele[1] + 1)):
            if shot_json[shot_id]["subtitle_id"] == [] : continue
            subtitle_list_in_shot = [old_subtitle_json[i]["sentence"] for i in range(shot_json[shot_id]["subtitle_id"][0], shot_json[shot_id]["subtitle_id"][1]+1)]
            new_subtitle_json[shot_id_idx * 3 + 0] = subtitle_list_in_shot[:len(subtitle_list_in_shot)//3 + (len(subtitle_list_in_shot) % 3 == 2) ]
            new_subtitle_json[shot_id_idx * 3 + 1] = subtitle_list_in_shot[len(subtitle_list_in_shot)//3 + (len(subtitle_list_in_shot) % 3 == 2)  : len(subtitle_list_in_shot)//3*2 + (len(subtitle_list_in_shot) % 3 == 1) + (len(subtitle_list_in_shot) % 3 == 2) ]
            new_subtitle_json[shot_id_idx * 3 + 2] = subtitle_list_in_shot[len(subtitle_list_in_shot)//3*2 + (len(subtitle_list_in_shot) % 3 == 1) + (len(subtitle_list_in_shot) % 3 == 2) : ]

        new_subtitle_file_path = Path(file_path, "subtitle.json")
        with open(new_subtitle_file_path, 'w') as f_new_subtitle_file:
            json.dump(new_subtitle_json, f_new_subtitle_file, indent=2)

        # generate and save info.json
        info_json = {}
        info_json["tt_id"] = tt_id
        info_json["img_id"] = [i*3 + j for i, shot_id in enumerate(range(img_list_ele[1] + 1 - img_list_ele[0])) for j in range(3)]
        info_json["shot_id"] = [shot_json[i]["shot_id"] for i in range(img_list_ele[0], img_list_ele[1] + 1) for j in range(3)]
        info_json["scene_id"] = [shot_json[i]["scene_id"] for i in range(img_list_ele[0], img_list_ele[1] + 1) for j in range(3)]

        info_json_path = Path(file_path, "info.json")
        with open(info_json_path, 'w') as f_info_json:
            json.dump(info_json, f_info_json, indent=2)
        
        

processing tt0047396 ...
tt0047396 has group 50 and total_num is 50
processing tt0048545 ...
tt0048545 has group 66 and total_num is 116
processing tt0049730 ...
tt0049730 has group 54 and total_num is 170
processing tt0052357 ...
tt0052357 has group 71 and total_num is 241
processing tt0056923 ...
tt0056923 has group 83 and total_num is 324
processing tt0061418 ...
tt0061418 has group 62 and total_num is 386
processing tt0061722 ...
tt0061722 has group 42 and total_num is 428
processing tt0062622 ...
tt0062622 has group 22 and total_num is 450
processing tt0063442 ...
tt0063442 has group 57 and total_num is 507
processing tt0065214 ...
tt0065214 has group 45 and total_num is 552
processing tt0065724 ...
tt0065724 has group 57 and total_num is 609
processing tt0066026 ...
tt0066026 has group 77 and total_num is 686
processing tt0067116 ...
tt0067116 has group 37 and total_num is 723
processing tt0068646 ...
tt0068646 has group 71 and total_num is 794
processing tt0070735 ...
tt0070735 